In [2]:
from first import FirstAPI
import pandas as pd
import json
from datetime import datetime, date
from dateutil.parser import parse
import hashlib
from process.track import process_track_data
from process.fixture import process_fixture_data
from process.race import process_race_data
from utils.logger import logger_1st
import traceback
from database.general import session_scope, FirstTrack, FirstFixture, FirstRace, FirstEntry, MapCourse, MapMeeting, MapRace, MapRunner

first_api = FirstAPI()









In [21]:
# track_data = first_api.get_tracks()

with open('downloads/sample_tracks_1.json', 'r') as f:
    track_data = json.load(f)


tracks = process_track_data(track_data)

pd.DataFrame(tracks)

,track_id,track_name,country_id,country_name,timezone,isdst,timezone_offset,code,video_code,track_type_id
0,3310,Ajax Downs,4,CA,EST,True,-14400,AJX,None,G
1,3162,Aqueduct,3,US,EST,True,-14400,AQU,None,G
2,3096,Assiniboia Downs,4,CA,CST,True,-18000,ASD,None,G
3,4282,Belmont At The Big A,3,US,EST,True,-14400,BAQ,None,G
4,3027,Belmont Park,3,US,EST,True,-14400,BEL,None,G
...,...,...,...,...,...,...,...,...,...,...
71,4112,Woodbine Mohawk Harness,4,CA,EST,True,-14400,WBS,None,T
72,3094,Woodbine TB,4,CA,EST,True,-14400,WO,None,G
73,3057,Yonkers Harness,3,US,EST,True,-14400,YRX,None,T
74,3434,Zanesville,3,US,,False,0,ZAN,None,T


In [ ]:


def upload_track_data(track: dict, overwrite: bool = False):
    try:
        with session_scope() as session:
            existing_track = session.query(FirstTrack).filter(FirstTrack.track_id == track['track_id']).first()
            if existing_track:
                if overwrite:
                    existing_track.track_name = track.get('track_name')
                    existing_track.country_id = track.get('country_id')
                    existing_track.country_name = track.get('country_name')
                    existing_track.timezone = track.get('timezone')
                    existing_track.isdst = track.get('isdst')
                    existing_track.timezone_offset = track.get('timezone_offset')
                    existing_track.code = track.get('code')
                    existing_track.video_code = track.get('video_code')
                    existing_track.track_type_id = track.get('track_type_id')

            else:
                new_track = FirstTrack(
                    track_id = track.get('track_id'),
                    track_name = track.get('track_name'),
                    country_id = track.get('country_id'),
                    country_name = track.get('country_name'),
                    timezone = track.get('timezone'),
                    isdst = track['isdst'],
                    timezone_offset = track['timezone_offset'],
                    code = track.get('code'),
                    video_code = track.get('video_code'),
                    track_type_id = track.get('track_type_id'),
                )
                session.add(new_track)
        return True
    except Exception as e:
        logger_1st.error(f'upload_track_data(): {e}')
        logger_1st.error(traceback.format_exc())
        return False


In [ ]:
# map these tracks

# # get all the tracks
# tracks = []
# with session_scope() as session:
#     result = session.query(FirstTrack).all()
#     for track in result:
#         tracks.append({
#             'track_id': track.track_id,
#             'track_name': track.track_name,
#             'country_name': track.country_name,
#         })

# # find the map_course_code
# for track in tracks:
#     with session_scope() as session:
#         result = session.query(MapCourse).filter(
#             MapCourse.full_name == track['track_name'],
#             MapCourse.country_code == track['country_name'],
#             ).first()
#         if result:
#             track['map_course_code'] = result.code
#         else:
#             track['map_course_code'] = None


# # upload the FirstTrack table
# for track in tracks:
#     if track['map_course_code'] is not None:
#         with session_scope() as session:
#             existing_track = session.query(FirstTrack).filter(FirstTrack.track_id == track['track_id']).first()
#             if existing_track:
#                 existing_track.tpd_course_id = track['map_course_code']



In [ ]:
# def get_tpd_meeting_id(track_id :str, fixture_date :date) -> int:
#     tpd_meeting_id = None
#     try:
#         track_id = str(track_id).strip()
#         track_id = None if track_id == '' else track_id
#         if track_id is not None and fixture_date is not None:
#             # get the tpd_course_id from the FirstTrack table
#             with session_scope() as session:
#                 result = session.query(FirstTrack.tpd_course_id).filter(FirstTrack.track_id == track_id).first()
#                 if result:
#                     tpd_course_id = result.tpd_course_id
#                 else:
#                     tpd_course_id = None
#                     logger_1st.error(f'get_tpd_meeting_id(): track_id: {track_id} not found in first_track table')

#             if tpd_course_id is not None:
#                 # get the tpd_meeting_id from the MapMeeting table
#                 with session_scope() as session:
#                     existing_meeting = session.query(MapMeeting.id).filter(
#                         MapMeeting.map_course_code == tpd_course_id, 
#                         MapMeeting.date == fixture_date).first()
#                     if existing_meeting:
#                         tpd_meeting_id = existing_meeting.id
#                     else:
#                         # create a new meeting
#                         new_meeting = MapMeeting(
#                             map_course_code = tpd_course_id,
#                             date = fixture_date,
#                         )
#                         session.add(new_meeting)
#                         session.flush()
#                         tpd_meeting_id = new_meeting.id
#             else:
#                 logger_1st.error(f'get_tpd_meeting_id(): track_id: {track_id} is not mapped')
#         else:
#             logger_1st.error(f'get_tpd_meeting_id(): track_id: {track_id}, fixture_date: {fixture_date} is not valid')

#     except Exception as e:
#         logger_1st.error(f'get_tpd_meeting_id(): track_id: {track_id}, fixture_date: {fixture_date}')
#         logger_1st.error(traceback.format_exc())
#     return tpd_meeting_id

In [ ]:
# def get_tpd_race_id(tpd_meeting_id :int, race_number :int, post_time :datetime) -> int:
#     tpd_race_id = None
#     try:
#         if tpd_meeting_id is not None and race_number is not None and post_time is not None:
#             with session_scope() as session:
#                 existing_race = session.query(MapRace.id).filter(
#                     MapRace.map_meeting_id == tpd_meeting_id,
#                     MapRace.race_number == race_number).first()
#                 if existing_race:
#                     tpd_race_id = existing_race.id
#                 else:
#                     new_race = MapRace(
#                         map_meeting_id = tpd_meeting_id,
#                         race_number = race_number,
#                         post_time = post_time,
#                     )
#                     session.add(new_race)
#                     session.flush()
#                     tpd_race_id = new_race.id
#         else:
#             logger_1st.error(f'get_tpd_race_id(): tpd_meeting_id: {tpd_meeting_id}, race_number: {race_number}, post_time: {post_time} is not valid')

#     except Exception as e:
#         logger_1st.error(f'get_tpd_race_id(): tpd_meeting_id: {tpd_meeting_id}, race_number: {race_number}, post_time: {post_time}')
#         logger_1st.error(traceback.format_exc())
#     return tpd_race_id

In [ ]:
# def get_tpd_runner_id(tpd_race_id :int, program_number :str) -> int:
#     tpd_runner_id = None
#     try:
#         program_number = str(program_number).strip() if program_number is not None else None
#         program_number = None if program_number == '' else program_number
#         if tpd_race_id is not None and program_number is not None:
#             with session_scope() as session:
#                 existing_runner = session.query(MapRunner.id).filter(
#                     MapRunner.map_race_id == tpd_race_id,
#                     MapRunner.runner_number == program_number).first()
#                 if existing_runner:
#                     tpd_runner_id = existing_runner.id
#                 else:
#                     new_runner = MapRunner(
#                         map_race_id = tpd_race_id,
#                         runner_number = program_number,
#                     )
#                     session.add(new_runner)
#                     session.flush()
#                     tpd_runner_id = new_runner.id
#         else:
#             logger_1st.error(f'get_tpd_runner_id(): tpd_race_id: {tpd_race_id}, program_number: {program_number} is not valid')
#     except Exception as e:
#         logger_1st.error(f'get_tpd_runner_id(): tpd_race_id: {tpd_race_id}, program_number: {program_number}')
#         logger_1st.error(traceback.format_exc())
#     return tpd_runner_id

In [ ]:
fixture_result = process_fixture_data(fixture_data)
fixtures = fixture_result['fixtures']
jockey_dict = fixture_result['jockey_dict']
trainer_dict = fixture_result['trainer_dict']
owner_dict = fixture_result['owner_dict']


In [4]:
jockey_dict.keys()

dict_keys(['59686', '57500', '146907', '164275', '146019', '172213', '58428', '151690', '172496', '163837', '173171', '95230', '174583', '162067', '83923', '155327', '124659', '156191', '81998', '160000', '109414', '159654', '172033', '34', '35', '165575', '1799', '162117', '133402', '3050655', '152730', '145057', '157701', '157929', '1175', '85561', '114441', '138717', '82321', '131537', '169358', '126107', '155352', '175685', '170593', '159338', '145572', '745', '134106', '57452', '903', '126315', '171801', '159279', '57770', '171717', '165566', '165827', '167274', '176811', '170203', '167807', '2664984', '131245', '152731', '172431', '176171', '113491', '56966', '159345', '137241', '146460', '149940', '126415', '1524', '108984', '3149828', '2789402', '105973', '124476', '101158', '107720', '174635', '0', '176172', '170371', '173152', '161703', '126417', '145189', '176907', '117699', '1137', '166268', '173136', '171648', '127764', '101342', '173945', '121913', '109036', '102032', '17

In [18]:
for key in jockey_dict.keys():
    print(key)

In [ ]:
required_jockeys = ['59686', '146019', '151690']

for jockey_id in required_jockeys:
    jockey_data = jockey_dict[jockey_id]
    print(jockey_data)



{'jockey_id': '59686', 'jockey_name': 'Luis E. Perez', 'old_jockey_id': '', 'old_jockey_name': ''}
{'jockey_id': '146019', 'jockey_name': 'Kevin Navarro', 'old_jockey_id': '', 'old_jockey_name': ''}
{'jockey_id': '151690', 'jockey_name': 'Andre Shivnarine Worrie', 'old_jockey_id': '', 'old_jockey_name': ''}


In [29]:
fixture.get('races')[0].get('entries', [])[0]

{'id': '20664570',
 'startNumber': '1',
 'programNumber': '1',
 'startPosition': '1',
 'scratch_indicator': '',
 'coupledIndicator': 0,
 'decoupledNumber': '1',
 'horse_id': '1081291',
 'name': 'Berning Beauty',
 'status': 1,
 'birthcountry': 'USA',
 'birthday': '2021-01-27',
 'age': 4,
 'saddleclothcolor': 'Red',
 'equipment': {'code': '', 'description': ''},
 'medication': {'code': 'L', 'description': 'Lasix'},
 'weight': {'value': '124',
  'overweight': None,
  'unit': 'lbs',
  'value_with_allowance': '124'},
 'breeder': 'Albert Fried Jr.',
 'owner': {'id': '735855',
  'name': 'Albert Fried, Jr.',
  'silks': 'Scarlet, Gold Ball, Lazy "F", Scarlet And Gold Halved Sleeves',
  'silksImage': 's110825/20250811xfl180001.png',
  'silksImageURL': 'https://silks.1stcontent.com/media/2025/08/11/14a01a170d11b7c881be09be8f363c6d.png'},
 'trainer': {'id': '148317', 'name': 'Charlton Baker'},
 'jockey': {'id': '59686',
  'name': 'Luis E. Perez',
  'oldJockeyID': '',
  'oldJockeyName': ''},
 'appr

In [16]:
data = first_api.get_horses(horse_id='1081291')
data

{'horse_id': '1081291',
 'externalId': '21000695',
 'name': 'Berning Beauty',
 'gender': {'value': 'g', 'details': 'Gelding'},
 'breed': {'value': 'TB', 'details': 'Thoroughbred'},
 'foaling': {'date': '2021-01-27', 'country': 'USA'},
 'color': {'value': 'B (Bay)', 'details': 'bay'},
 'breeder': 'Albert Fried Jr.',
 'pedigree': [{'id': '3011452',
   'type': 'Sire',
   'name': 'Bernardini',
   'foaling': {'date': '2003-03-23', 'area': 'KY'},
   'color': {'value': 'B', 'details': ''},
   'breed': {'value': 'TB', 'details': 'Thoroughbred'},
   'gender': {'value': 'H', 'details': 'Horse'}},
  {'id': '9009646',
   'type': 'Dam',
   'name': "The Giant's Beauty",
   'foaling': {'date': '2009-03-18', 'area': 'NY'},
   'color': {'value': 'CH', 'details': ''},
   'breed': {'value': 'TB', 'details': 'Thoroughbred'},
   'gender': {'value': 'M', 'details': 'Mare'}},
  {'id': '93004603',
   'type': 'SireDam',
   'name': 'Cara Rafaela',
   'foaling': {'date': '1993-03-30', 'area': 'KY'},
   'color': 